In [1]:
import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.multitest import multipletests
from scipy.stats import norm

# Panel a-b

In [6]:
csv_path = '/home/sarth/rootdir/workdir/projects/Paper_Data_Latency/Revised_Statistical_Testing/Figure02'

def process(varname = 'NSE', min_max = [-10, 1], verbose = False):
    df = pd.read_csv(f'{csv_path}/diag_notdiag_{varname}.csv', index_col=0)

    # Quick checks
    df = df.fillna(min_max[0]).reset_index(drop=True)
    if verbose: 
        print("Number of Rows with NaNs?")
        print(df.isna().sum())
        print(f"Shape: {df.shape}")
        print("Counts per region:")
        print(df["huc"].value_counts().sort_index())
    df["diff"] = df["diagonal"] - df["not_diagonal"]

    results = {}

    ###############################################################

    # One-sided WSR (H1: diagonal > non-diagonal)
    w = stats.wilcoxon(df["diagonal"], df["not_diagonal"], alternative="greater", zero_method="wilcox")
    if verbose: print({"Wilcoxon_W": w.statistic, "pvalue": w.pvalue})
    results["Wilcoxon_W"] = w.statistic
    results["Wilcoxon_p"] = w.pvalue

    ###############################################################

    # Paired effect sizes: rank-biserial and CLES
    wins   = int((df["diff"] > 0).sum())
    losses = int((df["diff"] < 0).sum())
    ties   = int((df["diff"] == 0).sum())

    # Rank-biserial correlation (paired; ignoring ties in denom = common practice)
    r_rb = (wins - losses) / (wins + losses) if (wins + losses) > 0 else np.nan

    # Common-Language Effect Size: P(diagonal > non-diagonal)
    cles = wins / (wins + losses) if (wins + losses) > 0 else np.nan
    if verbose: print({"wins": wins, "losses": losses, "ties": ties, "rank_biserial": r_rb, "CLES": cles})
    results["wins"] = wins
    results["losses"] = losses
    results["ties"] = ties
    results["rank_biserial"] = r_rb
    results["CLES"] = cles

    ###############################################################

    rng = np.random.default_rng(42)
    H = df["huc"].unique()
    B = 5000  # increase if you want tighter CIs; keep an eye on runtime

    boot_med = np.empty(B)
    for b in range(B):
        h_samp = rng.choice(H, size=len(H), replace=True) # sample HUCs w/ replacement
        d_b = pd.concat([df.loc[df["huc"]==h, "diff"] for h in h_samp])
        boot_med[b] = np.median(d_b.to_numpy())

    median_diff = float(np.median(df["diff"]))
    ci_med = (np.percentile(boot_med, 2.5), np.percentile(boot_med, 97.5))
    if verbose: print({"median_diff": median_diff, "median_diff_CI95": ci_med})
    results["median_diff"] = median_diff
    results["median_diff_CI95"] = ci_med

    ###############################################################

    # Per-HUC paired WSR (one-sided)
    rows = []
    for h, g in df.groupby("huc"):
        res = stats.wilcoxon(g["diagonal"], g["not_diagonal"],
                            alternative="greater", zero_method="wilcox")
        med_delta = float(np.median(g["diff"]))
        rows.append({"huc": h, "n": len(g), "p": res.pvalue, "med_delta": med_delta})

    per_huc = pd.DataFrame(rows).sort_values("huc").reset_index(drop=True)

    # BH-FDR within the family of per-HUC tests
    per_huc["q_BH"] = multipletests(per_huc["p"].to_numpy(), method="fdr_bh")[1]
    if verbose: print(per_huc)

    # Equal-HUC Stouffer combine (direction from median difference)
    p = per_huc["p"].clip(lower=np.finfo(float).tiny).to_numpy()
    sign = np.sign(per_huc["med_delta"].to_numpy())
    Zs = norm.isf(p) * sign
    w = np.ones_like(Zs)  # equal-HUC weighting (avoid big HUC domination)
    Z_comb = (w * Zs).sum() / np.sqrt((w**2).sum())
    p_comb = norm.sf(Z_comb)
    if verbose: print({"Stouffer_Z_equalHUC": Z_comb, "pvalue": p_comb})

    results["per_huc"] = per_huc
    results["Stouffer_Z_equalHUC"] = Z_comb
    results["Stouffer_p_equalHUC"] = p_comb

    ################################################################

    x = df["diagonal"].to_numpy()
    y = df["not_diagonal"].to_numpy()

    # Brunner–Munzel (SciPy)
    bm_two = stats.brunnermunzel(x, y, alternative="two-sided")
    bm_greater = stats.brunnermunzel(x, y, alternative="greater")
    if verbose: print({"BM_two_sided": (bm_two.statistic, bm_two.pvalue), "BM_greater": (bm_greater.statistic, bm_greater.pvalue)})

    results["BM_two_sided"] = (bm_two.statistic, bm_two.pvalue)
    results["BM_greater"] = (bm_greater.statistic, bm_greater.pvalue)

    ################################################################

    # Cliff's delta via Mann–Whitney U (efficient): δ = 2U/(mn) - 1
    def cliffs_delta_via_u(a, b):
        u = stats.mannwhitneyu(a, b, alternative="two-sided").statistic
        m, n = len(a), len(b)
        return (2*u)/(m*n) - 1

    delta_hat = cliffs_delta_via_u(x, y)

    # HUC-cluster bootstrap CI for δ (fast via U in each bootstrap)
    rng = np.random.default_rng(123)
    H = df["huc"].unique()
    B = 2000
    boot_delta = np.empty(B)

    for b in range(B):
        h_samp = rng.choice(H, size=len(H), replace=True)
        df_b = pd.concat([df[df["huc"]==h] for h in h_samp], ignore_index=True)
        boot_delta[b] = cliffs_delta_via_u(df_b["diagonal"].to_numpy(),
                                        df_b["not_diagonal"].to_numpy())

    ci_delta = (np.percentile(boot_delta, 2.5), np.percentile(boot_delta, 97.5))
    if verbose: print({"cliffs_delta": delta_hat, "cliffs_delta_CI95": ci_delta})

    results["cliffs_delta"] = delta_hat
    results["cliffs_delta_CI95"] = ci_delta

    ################################################################

    def q1(a): return np.percentile(a, 25.0)

    q1_diag = q1(x)
    q1_nondiag = q1(y)
    q1_shift = q1_diag - q1_nondiag

    # Cluster bootstrap CI for Q1 shift
    rng = np.random.default_rng(99)
    boot_q1 = np.empty(B)
    for b in range(B):
        h_samp = rng.choice(H, size=len(H), replace=True)
        df_b = pd.concat([df[df["huc"]==h] for h in h_samp], ignore_index=True)
        boot_q1[b] = q1(df_b["diagonal"]) - q1(df_b["not_diagonal"])

    ci_q1 = (np.percentile(boot_q1, 2.5), np.percentile(boot_q1, 97.5))
    if verbose: print({"Q1_shift": q1_shift, "Q1_shift_CI95": ci_q1})

    results["Q1_shift"] = q1_shift
    results["Q1_shift_CI95"] = ci_q1

    ###############################################################

    # results = {
    #     "Wilcoxon_W": w.statistic,
    #     "Wilcoxon_p": w.pvalue,
    #     "wins": wins,
    #     "losses": losses,
    #     "ties": ties,
    #     "rank_biserial": r_rb,
    #     "CLES": cles,
    #     "median_diff": median_diff,
    #     "median_diff_CI95": ci_med,
    #     "per_huc": per_huc,
    #     "Stouffer_Z_equalHUC": Z_comb,
    #     "Stouffer_p_equalHUC": p_comb,
    #     "BM_two_sided": (bm_two.statistic, bm_two.pvalue),
    #     "BM_greater": (bm_greater.statistic, bm_greater.pvalue),
    #     "cliffs_delta": delta_hat,
    #     "cliffs_delta_CI95": ci_delta,
    #     "Q1_shift": q1_shift,
    #     "Q1_shift_CI95": ci_q1
    # }
    return results

In [4]:
nse_results = process(varname='NSE', min_max=[-10, 1], verbose=False)
nse_results

{'Wilcoxon_W': np.float64(74258.5),
 'Wilcoxon_p': np.float64(5.525707413019561e-65),
 'wins': 383,
 'losses': 2,
 'ties': 10,
 'rank_biserial': 0.9896103896103896,
 'CLES': 0.9948051948051948,
 'median_diff': 0.4,
 'median_diff_CI95': (np.float64(0.36), np.float64(0.43000000000000005)),
 'per_huc':     huc   n             p  med_delta          q_BH
 0     1  12  4.882812e-04      0.295  7.990057e-04
 1     2  39  3.863805e-08      0.480  1.390970e-07
 2     3  45  2.583848e-09      0.420  1.550309e-08
 3     4  16  7.332615e-04      0.355  1.099892e-03
 4     5  26  6.122912e-06      0.280  1.224582e-05
 5     6  10  9.765625e-04      0.195  1.352163e-03
 6     7  26  1.490116e-08      0.420  6.705523e-08
 7     8  10  1.953125e-03      0.195  2.511161e-03
 8     9   5  3.125000e-02      0.560  3.515625e-02
 9    10  52  2.568169e-10      0.495  2.312860e-09
 10   11  26  4.135161e-06      0.365  9.327541e-06
 11   12  28  4.145574e-06      0.385  9.327541e-06
 12   13   4  6.250000e-

In [7]:
f1_results = process(varname='F1', min_max=[0, 1], verbose=False)
f1_results

{'Wilcoxon_W': np.float64(73242.0),
 'Wilcoxon_p': np.float64(2.6241376581169765e-54),
 'wins': 362,
 'losses': 30,
 'ties': 3,
 'rank_biserial': 0.8469387755102041,
 'CLES': 0.923469387755102,
 'median_diff': 0.09000000000000008,
 'median_diff_CI95': (np.float64(0.08000000000000007),
  np.float64(0.10000000000000009)),
 'per_huc':     huc   n             p  med_delta          q_BH
 0     1  12  4.882812e-04      0.085  8.789062e-04
 1     2  39  3.688295e-08      0.120  1.659733e-07
 2     3  45  1.786203e-08      0.120  1.408389e-07
 3     4  16  3.575223e-03      0.075  4.950309e-03
 4     5  26  3.474452e-04      0.065  6.948904e-04
 5     6  10  9.765625e-04      0.100  1.464844e-03
 6     7  26  1.810315e-04      0.080  4.073209e-04
 7     8  10  1.562500e-02      0.080  1.875000e-02
 8     9   5  3.125000e-02      0.080  3.515625e-02
 9    10  52  2.347315e-08      0.100  1.408389e-07
 10   11  26  6.595688e-05      0.090  2.374448e-04
 11   12  28  8.425156e-05      0.085  2.52

# Panel c-d

In [8]:
# -------------------------------
# 0) Utility helpers
# -------------------------------

def p_stars(p):
    if p < 1e-3: return '***'
    if p < 1e-2: return '**'
    if p < 5e-2: return '*'
    return 'ns'

def fmt_p(p):
    if p == 0 or (np.isfinite(p) and p < 1e-300):  # handle underflow
        return "≈0"
    return f"{p:.2e}"

def cluster_bootstrap_ci(values, clusters, stat_fn=np.median, B=5000, seed=42):
    """
    Block/cluster bootstrap: resample unique cluster labels with replacement,
    collect all rows from sampled clusters, compute stat_fn on the pooled vector.
    Returns (lo, hi) 95% CI.
    """
    rng = np.random.default_rng(seed)
    values = np.asarray(values)
    clusters = np.asarray(clusters)
    H = np.unique(clusters)
    boot = np.empty(B, dtype=float)
    for b in range(B):
        h_samp = rng.choice(H, size=len(H), replace=True)
        idx = np.isin(clusters, h_samp)
        boot[b] = stat_fn(values[idx])
    return (np.nanpercentile(boot, 2.5), np.nanpercentile(boot, 97.5))

def cliffs_delta_via_u(a, b):
    """
    Fast Cliff's delta via Mann–Whitney U:
    delta = 2U/(mn) - 1
    """
    a = np.asarray(a); b = np.asarray(b)
    u = stats.mannwhitneyu(a, b, alternative="two-sided").statistic
    m, n = len(a), len(b)
    return (2*u)/(m*n) - 1

def stouffer_equal_huc(pvals, signs):
    """
    Combine one-sided p-values across HUCs with equal HUC weights.
    'signs' derived from direction (e.g., median delta).
    """
    p = np.clip(np.asarray(pvals, dtype=float), np.finfo(float).tiny, 1.0)
    Z = norm.isf(p) * np.sign(signs)
    w = np.ones_like(Z)
    Zc = (w * Z).sum() / np.sqrt((w**2).sum())
    pc = norm.sf(Zc)
    return Zc, pc

def paired_effect_sizes(diff):
    """
    Rank-biserial (paired) and CLES from wins/losses/ties.
    """
    diff = np.asarray(diff)
    wins   = int((diff > 0).sum())
    losses = int((diff < 0).sum())
    ties   = int((diff == 0).sum())
    denom = wins + losses
    if denom == 0:
        r_rb = np.nan
        cles = np.nan
    else:
        r_rb = (wins - losses) / denom
        cles = wins / denom
    return wins, losses, ties, r_rb, cles

# CDF/PDF curve metrics
def trapezoid_integral(x, y): return np.trapz(y, x)
def ks_distance(cdf_a, cdf_b): return np.max(np.abs(cdf_a - cdf_b))
def cvm_distance(x, cdf_a, cdf_b): return trapezoid_integral(x, (cdf_a - cdf_b)**2)
def wasserstein1(x, cdf_a, cdf_b): return trapezoid_integral(x, np.abs(cdf_a - cdf_b))
def hellinger(x, pdf_a, pdf_b): return np.sqrt(0.5 * trapezoid_integral(x, (np.sqrt(pdf_a) - np.sqrt(pdf_b))**2))

def dominance_violations(cdf_cont, cdf_reg):
    """
    For first-order dominance (higher is better), we want F_cont(x) <= F_reg(x) for all x.
    Report max positive violation and fraction of violating grid points.
    """
    diff = cdf_cont - cdf_reg
    return float(np.max(np.maximum(diff, 0.0))), float(np.mean(diff > 0.0))

def invert_cdf(x_grid, cdf, p):
    return float(np.interp(p, cdf, x_grid, left=x_grid[0], right=x_grid[-1]))

def quantile_shifts_from_cdfs(x_grid, cdf_c, cdf_r, probs=(0.10,0.25,0.50,0.75,0.90)):
    out = []
    for pr in probs:
        qc = invert_cdf(x_grid, cdf_c, pr)
        qr = invert_cdf(x_grid, cdf_r, pr)
        out.append((pr, qc - qr, qc, qr))
    return pd.DataFrame(out, columns=["p","DeltaQ","Q_cont","Q_reg"])

In [9]:
# -------------------------------
# 1) Load data & pair basins
# -------------------------------
csv_path = '/home/sarth/rootdir/workdir/projects/Paper_Data_Latency/Revised_Statistical_Testing/Figure02'
cont = pd.read_csv(f"{csv_path}/continental.csv")          # columns: huc, NSE, F1
reg  = pd.read_csv(f"{csv_path}/median_regional.csv")      # columns: huc, NSE, F1

cont = cont.rename(columns={"prediction_huc": "huc"})
reg = reg.rename(columns={"prediction_huc": "huc"})

# Fill NaNs in NSE column with -10, and in F1 column with 0
cont['NSE'] = cont['NSE'].fillna(-10)
reg['NSE'] = reg['NSE'].fillna(-10)
cont['F1'] = cont['F1'].fillna(0)
reg['F1'] = reg['F1'].fillna(0)

In [10]:
# Pairing strategy:
# If a unique basin identifier exists in both files (e.g., 'basin_id'), merge on it.
# Otherwise, we create a within-HUC index and merge on (huc, idx). This assumes
# the ordering within each HUC is consistent across both files.

if "basin_id" in cont.columns and "basin_id" in reg.columns:
    df = cont.merge(reg, on=["basin_id","huc"], suffixes=("_C", "_R"))
else:
    cont = cont.copy()
    reg  = reg.copy()
    cont["idx_within_huc"] = cont.groupby("huc").cumcount()
    reg["idx_within_huc"]  = reg.groupby("huc").cumcount()
    df = cont.merge(reg, on=["huc","idx_within_huc"], suffixes=("_C", "_R"))
    # If you know files are in identical row order, you could simply join by index instead.

# Sanity checks
assert df[["NSE_C","NSE_R","F1_C","F1_R","huc"]].notna().all().all(), "NaNs present after merge"
print("Paired rows:", df.shape[0], "Unique HUCs:", df['huc'].nunique())

# -------------------------------
# 2) Primary paired inference (per metric)
# -------------------------------

def paired_wilcoxon_block(df, metric_col_C, metric_col_R, huc_col="huc", one_sided=True, B=5000, seed=42):
    C = df[metric_col_C].to_numpy()
    R = df[metric_col_R].to_numpy()
    H = df[huc_col].to_numpy()
    diff = C - R

    # Wilcoxon signed-rank (one-sided C > R)
    res = stats.wilcoxon(C, R, alternative=("greater" if one_sided else "two-sided"), zero_method="wilcox")
    W, p_w = res.statistic, res.pvalue

    # Paired effect sizes
    wins, losses, ties, r_rb, cles = paired_effect_sizes(diff)

    # Median paired difference + HUC-cluster CI
    med_diff = float(np.median(diff))
    ci_lo, ci_hi = cluster_bootstrap_ci(diff, clusters=H, stat_fn=np.median, B=B, seed=seed)

    return {
        "W": float(W), "p_wilcoxon": float(p_w),
        "wins": wins, "losses": losses, "ties": ties,
        "median_diff": med_diff, "median_diff_CI95": (float(ci_lo), float(ci_hi)),
        "r_rb": float(r_rb), "CLES": float(cles),
    }

# -------------------------------
# 3) Per-HUC paired tests + BH-FDR + Stouffer
# -------------------------------

def per_huc_paired(df, metric_col_C, metric_col_R, huc_col="huc"):
    rows = []
    for h, g in df.groupby(huc_col):
        if g.shape[0] < 2:
            rows.append({"huc": h, "n": len(g), "p": np.nan, "med_delta": float(np.median(g[metric_col_C]-g[metric_col_R]))})
            continue
        res = stats.wilcoxon(g[metric_col_C], g[metric_col_R], alternative="greater", zero_method="wilcox")
        rows.append({"huc": h, "n": len(g), "p": float(res.pvalue), "med_delta": float(np.median(g[metric_col_C]-g[metric_col_R]))})
    per = pd.DataFrame(rows).sort_values("huc").reset_index(drop=True)
    # BH-FDR within metric
    mask = per["p"].notna()
    per.loc[mask, "q_BH"] = multipletests(per.loc[mask, "p"].to_numpy(), method="fdr_bh")[1]
    per["q_BH"] = per["q_BH"].astype(float)
    # Stouffer (equal HUC weights)
    valid = per["p"].notna()
    Zc, pc = stouffer_equal_huc(per.loc[valid, "p"].to_numpy(), per.loc[valid, "med_delta"].to_numpy())
    return per, Zc, pc

# -------------------------------
# 4) Distributional lens: BM + Cliff's δ + Q1 shift (with cluster CIs)
# -------------------------------

def distributional_lens(df, metric_col_C, metric_col_R, huc_col="huc", B=5000, seed=123):
    C = df[metric_col_C].to_numpy()
    R = df[metric_col_R].to_numpy()
    H = df[huc_col].to_numpy()

    # Brunner–Munzel (one-sided C > R)
    bm = stats.brunnermunzel(C, R, alternative="greater")

    # Cliff's delta (point estimate)
    delta_hat = cliffs_delta_via_u(C, R)

    # Cluster CI for delta
    rng = np.random.default_rng(seed)
    Huniq = np.unique(H)
    boot_d = np.empty(B)
    for b in range(B):
        hs = rng.choice(Huniq, size=len(Huniq), replace=True)
        idx = np.isin(H, hs)
        boot_d[b] = cliffs_delta_via_u(C[idx], R[idx])
    d_lo, d_hi = np.percentile(boot_d, 2.5), np.percentile(boot_d, 97.5)

    # Q1 (25th percentile) shift + cluster CI
    def q1_shift(valsC, valsR): return np.percentile(valsC, 25.0) - np.percentile(valsR, 25.0)
    q1 = q1_shift(C, R)

    boot_q1 = np.empty(B)
    for b in range(B):
        hs = rng.choice(Huniq, size=len(Huniq), replace=True)
        idx = np.isin(H, hs)
        boot_q1[b] = q1_shift(C[idx], R[idx])
    q_lo, q_hi = np.percentile(boot_q1, 2.5), np.percentile(boot_q1, 97.5)

    return {
        "bm_stat": float(bm.statistic), "bm_p": float(bm.pvalue),
        "delta": float(delta_hat), "delta_CI95": (float(d_lo), float(d_hi)),
        "Q1_shift": float(q1), "Q1_shift_CI95": (float(q_lo), float(q_hi)),
    }

# -------------------------------
# 5) CDF/PDF curve metrics (descriptive) from files
# -------------------------------

def curve_metrics_from_files(nse_curve_csv="NSE_pdf_cdf.csv", f1_curve_csv="F1_pdf_cdf.csv"):
    nse_curves = pd.read_csv(nse_curve_csv)
    f1_curves  = pd.read_csv(f1_curve_csv)

    # NSE
    x = nse_curves["grid"].to_numpy()
    Fn_c = nse_curves["NSE_cdf_continental"].to_numpy()
    Fn_r = nse_curves["NSE_cdf_regional"].to_numpy()
    pn_c = nse_curves["NSE_pdf_continental"].to_numpy()
    pn_r = nse_curves["NSE_pdf_regional"].to_numpy()

    nse_curve = {
        "KS": ks_distance(Fn_c, Fn_r),
        "CvM": cvm_distance(x, Fn_c, Fn_r),
        "W1": wasserstein1(x, Fn_c, Fn_r),
        "Hellinger": hellinger(x, pn_c, pn_r),
        "Dom.max_violation": dominance_violations(Fn_c, Fn_r)[0],
        "Dom.frac_violation": dominance_violations(Fn_c, Fn_r)[1],
        "Qshifts": quantile_shifts_from_cdfs(x, Fn_c, Fn_r)
    }

    # F1
    x = f1_curves["grid"].to_numpy()
    F1_c = f1_curves["F1_cdf_continental"].to_numpy()
    F1_r = f1_curves["F1_cdf_regional"].to_numpy()
    pf_c = f1_curves["F1_pdf_continental"].to_numpy()
    pf_r = f1_curves["F1_pdf_regional"].to_numpy()

    f1_curve = {
        "KS": ks_distance(F1_c, F1_r),
        "CvM": cvm_distance(x, F1_c, F1_r),
        "W1": wasserstein1(x, F1_c, F1_r),
        "Hellinger": hellinger(x, pf_c, pf_r),
        "Dom.max_violation": dominance_violations(F1_c, F1_r)[0],
        "Dom.frac_violation": dominance_violations(F1_c, F1_r)[1],
        "Qshifts": quantile_shifts_from_cdfs(x, F1_c, F1_r)
    }

    return nse_curve, f1_curve

# -------------------------------
# 6) Run pipeline for NSE and F1
# -------------------------------

def run_metric(df, metric, B_ci=5000):
    C_col = f"{metric}_C"
    R_col = f"{metric}_R"

    # Paired inference
    paired = paired_wilcoxon_block(df, C_col, R_col, huc_col="huc", one_sided=True, B=B_ci)

    # Per-HUC drilldown
    per_huc, Zc, pc = per_huc_paired(df, C_col, R_col, huc_col="huc")
    hucs_sig = int((per_huc["q_BH"] < 0.05).sum())

    # Distributional lens
    dist = distributional_lens(df, C_col, R_col, huc_col="huc", B=B_ci)

    # Assemble summary row
    summary = {
        "Metric": metric,
        "N (basins)": df.shape[0],
        "Wins/Losses/Ties": f'{paired["wins"]} / {paired["losses"]} / {paired["ties"]}',
        "Median Δ [95% CI]": f'{paired["median_diff"]:+.3f} [{paired["median_diff_CI95"][0]:.3f}, {paired["median_diff_CI95"][1]:.3f}]',
        "Wilcoxon W, p": f'{paired["W"]:,.1f}; p={fmt_p(paired["p_wilcoxon"])} {p_stars(paired["p_wilcoxon"])}',
        "Rank-biserial r_rb": f'{paired["r_rb"]:.3f}',
        "CLES": f'{paired["CLES"]:.3f}',
        "BM p (one-sided)": f'{dist["bm_stat"]} p={fmt_p(dist["bm_p"])} {p_stars(dist["bm_p"])}',
        "Cliff’s δ [95% CI]": f'{dist["delta"]:.3f} [{dist["delta_CI95"][0]:.3f}, {dist["delta_CI95"][1]:.3f}]',
        "Q1 shift [95% CI]": f'{dist["Q1_shift"]:+.3f} [{dist["Q1_shift_CI95"][0]:.3f}, {dist["Q1_shift_CI95"][1]:.3f}]',
        "HUCs q<0.05 / 18": f'{hucs_sig} / {df["huc"].nunique()}',
        "Stouffer Z, p": f'{Zc:.2f}; p={fmt_p(pc)} {p_stars(pc)}',
    }
    return summary, per_huc

Paired rows: 395 Unique HUCs: 18


In [11]:
# Execute
summary_nse, per_huc_nse = run_metric(df, "NSE", B_ci=3000)
summary_f1,  per_huc_f1  = run_metric(df, "F1",  B_ci=3000)

In [12]:
summary_tbl = pd.DataFrame([summary_nse, summary_f1])
print("\n=== Main summary (paired + distributional) ===")
print(summary_tbl.to_markdown(index=False))


=== Main summary (paired + distributional) ===
| Metric   |   N (basins) | Wins/Losses/Ties   | Median Δ [95% CI]     | Wilcoxon W, p            |   Rank-biserial r_rb |   CLES | BM p (one-sided)                    | Cliff’s δ [95% CI]   | Q1 shift [95% CI]     | HUCs q<0.05 / 18   | Stouffer Z, p         |
|:---------|-------------:|:-------------------|:----------------------|:-------------------------|---------------------:|-------:|:------------------------------------|:---------------------|:----------------------|:-------------------|:----------------------|
| NSE      |          395 | 386 / 9 / 0        | +0.400 [0.386, 0.416] | 74,769.0; p=6.74e-56 *** |                0.954 |  0.977 | -33.669698809588816 p=2.13e-129 *** | 0.800 [0.771, 0.828] | +0.754 [0.697, 0.809] | 16 / 18            | 16.31; p=4.29e-60 *** |
| F1       |          395 | 363 / 31 / 1       | +0.094 [0.087, 0.101] | 73,713.0; p=9.93e-54 *** |                0.843 |  0.921 | -13.01646020684048 p=6.52e-34 *** 

In [13]:
print("\n=== Per-HUC (NSE) with BH-FDR ===")
print(per_huc_nse.to_markdown(index=False))


=== Per-HUC (NSE) with BH-FDR ===
|   huc |   n |           p |   med_delta |        q_BH |
|------:|----:|------------:|------------:|------------:|
|     1 |  12 | 0.0170898   |    0.313206 | 0.0205078   |
|     2 |  39 | 1.37952e-08 |    0.454908 | 4.47035e-08 |
|     3 |  45 | 2.84217e-14 |    0.421025 | 5.11591e-13 |
|     4 |  16 | 0.00257874  |    0.426015 | 0.00357056  |
|     5 |  26 | 1.34706e-05 |    0.370727 | 3.0309e-05  |
|     6 |  10 | 0.000976562 |    0.294183 | 0.00146484  |
|     7 |  26 | 1.49012e-08 |    0.428108 | 4.47035e-08 |
|     8 |  10 | 0.000976562 |    0.256916 | 0.00146484  |
|     9 |   5 | 0.03125     |    0.606978 | 0.0351562   |
|    10 |  52 | 3.29395e-09 |    0.417784 | 1.97637e-08 |
|    11 |  26 | 1.49012e-08 |    0.32731  | 4.47035e-08 |
|    12 |  28 | 0.000108805 |    0.386444 | 0.000217609 |
|    13 |   4 | 0.0625      |    0.353254 | 0.0661765   |
|    14 |   5 | 0.3125      |    0.310917 | 0.3125      |
|    15 |  12 | 0.000244141 |    0.50

In [14]:
print("\n=== Per-HUC (F1) with BH-FDR ===")
print(per_huc_f1.to_markdown(index=False))


=== Per-HUC (F1) with BH-FDR ===
|   huc |   n |           p |   med_delta |        q_BH |
|------:|----:|------------:|------------:|------------:|
|     1 |  12 | 0.000488281 |   0.0847923 | 0.000878906 |
|     2 |  39 | 2.49202e-10 |   0.11823   | 2.24281e-09 |
|     3 |  45 | 4.80327e-12 |   0.12122   | 8.64588e-11 |
|     4 |  16 | 0.0288391   |   0.0890908 | 0.0370789   |
|     5 |  26 | 6.29425e-05 |   0.0859306 | 0.000141621 |
|     6 |  10 | 0.00488281  |   0.0884298 | 0.00676082  |
|     7 |  26 | 4.47035e-08 |   0.0950203 | 1.82588e-07 |
|     8 |  10 | 0.000976562 |   0.0884488 | 0.00146484  |
|     9 |   5 | 0.21875     |   0.0320068 | 0.21875     |
|    10 |  52 | 5.07189e-08 |   0.0824341 | 1.82588e-07 |
|    11 |  26 | 1.04308e-07 |   0.103985  | 3.12924e-07 |
|    12 |  28 | 0.000212483 |   0.0902088 | 0.000424966 |
|    13 |   4 | 0.0625      |   0.113079  | 0.0661765   |
|    14 |   5 | 0.0625      |   0.0833858 | 0.0661765   |
|    15 |  12 | 0.000732422 |   0.0753

In [15]:
# -------------------------------
# 7) Curve-level distances (descriptive; aligns with your PDF/CDF panels)
# -------------------------------
nse_curve, f1_curve = curve_metrics_from_files(f"{csv_path}/NSE_cdf_pdf.csv", f"{csv_path}/F1_cdf_pdf.csv")

curve_tbl = pd.DataFrame([
    {"Metric": "NSE", "KS": nse_curve["KS"], "CvM": nse_curve["CvM"], "W1": nse_curve["W1"],
     "Hellinger": nse_curve["Hellinger"], "Dom.max_violation": nse_curve["Dom.max_violation"],
     "Dom.frac_violation": nse_curve["Dom.frac_violation"]},
    {"Metric": "F1", "KS": f1_curve["KS"], "CvM": f1_curve["CvM"], "W1": f1_curve["W1"],
     "Hellinger": f1_curve["Hellinger"], "Dom.max_violation": f1_curve["Dom.max_violation"],
     "Dom.frac_violation": f1_curve["Dom.frac_violation"]},
])
print("\n=== Curve distances (descriptive) ===")
print(curve_tbl.to_markdown(index=False))


=== Curve distances (descriptive) ===
| Metric   |       KS |       CvM |        W1 |   Hellinger |   Dom.max_violation |   Dom.frac_violation |
|:---------|---------:|----------:|----------:|------------:|--------------------:|---------------------:|
| NSE      | 0.624051 | 0.21472   | 0.567856  |         nan |          0          |                 0    |
| F1       | 0.421519 | 0.0256405 | 0.0999489 |         nan |          0.00253165 |                 0.06 |


/tmp/ipykernel_73250/1783315693.py:73: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  def trapezoid_integral(x, y): return np.trapz(y, x)
/tmp/ipykernel_73250/1783315693.py:77: RuntimeWarning: invalid value encountered in sqrt
  def hellinger(x, pdf_a, pdf_b): return np.sqrt(0.5 * trapezoid_integral(x, (np.sqrt(pdf_a) - np.sqrt(pdf_b))**2))
